# Recommendation Systems - Lab

## Introduction

Now that you've gotten an introduction to collaborative filtering and recommendation systems, it's time to put your skills to test and attempt to build a recommendation system for a real world dataset! For this exercise, you'll be using a dataset regarding the book reviews on the Amazon marketplace. While the previous lesson focused on user-based recommendation systems, you'll apply a parallel process for an item-based recommendation system to recommend similar books at the bottom of the product page.

## Objectives

You will be able to:
* Implement a recommendation system on a real world dataset

## Load the Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
df = pd.read_csv("books_data.edgelist", delimiter=" ", names=["book_1", "book_2", "weight"])
df.head()

,book_1,book_2,weight
0,0827229534,0804215715,0.7
1,0827229534,156101074X,0.5
2,0827229534,0687023955,0.8
3,0827229534,0687074231,0.8
4,0827229534,082721619X,0.7


## Load the MetaData

In [9]:
col_names = ["Id", "ASIN","Title","Categories",
             "Group", "SalesRank", "TotalReviews",
             "AvgRating", "DegreeCentrality", "ClusteringCoeff"]

books = pd.read_csv("books_meta.txt", delimiter="\t")
books.head()

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,clergi sermon subject religion preach spiritu ...,Book,396585,2,5.0,8,0.80
1,2,0738700797,Candlemas: Feast of Flames,subject witchcraft earth religion spiritu base...,Book,168596,12,4.5,9,0.85
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,general hobbi subject craft home garden book,Book,1270652,1,5.0,0,0.00
3,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,spiritu translat commentari christian book gui...,Book,631289,1,4.0,6,0.79
4,5,1577943082,Prayers That Avail Much for Business: Executive,subject religion spiritu busi christian live w...,Book,455160,0,0.0,4,1.00


## Select Books to Test Your Recommender On

Select a small subset of books that you are interested in generating recommendations for. 

In [18]:
HP=books[books["Title"].str.contains("Harry Potter")]
HP

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
1641,2299,0828819009,Harry Potter e la Pietra Filosofale (Italian E...,adventur general subject author rowl j illustr...,Book,983026,0,0.0,0,0.00
3381,4739,0939173379,Harry Potter and the Goblet of Fire (Book 4),literatur j seri book r potter k fiction gener...,Book,868892,4921,5.0,4,1.00
4332,6051,1575580675,Pokemon & Harry Potter: A Fatal Attraction,literatur parent popular seri book pokémon pot...,Book,1097841,30,1.5,0,0.00
4385,6130,0807220280,Harry Potter and the Order of the Phoenix (Boo...,literatur j seri tape book format r potter fic...,Book,2135,5539,4.5,4,1.00
5740,8086,0740715771,Harry Potter and the Sorcerer's Stone 2002 Cal...,NaN,Book,340296,0,0.0,3,0.83
6029,8504,0972393617,Ultimate Unofficial Guide to the Mysteries of ...,adventur general subject literatur magic ficti...,Book,4468,107,4.0,16,0.78
10493,15011,5353005791,Garri Potter i Kubok ognia (Harry Potter and t...,adventur general french subject author rowl j ...,Book,160581,7,4.0,3,0.00
11656,16592,031226481X,We Love Harry Potter!,adventur nonfict general subject seri paperbac...,Book,848383,23,1.5,0,0.00
11659,16597,1859846661,Irresistible Rise of Harry Potter,literatur parent seri book potter guid fiction...,Book,577640,3,3.0,4,0.43
12255,17436,0740717561,Harry Potter 2002 Calendar,NaN,Book,1210904,0,0.0,0,0.00


## Generate Recommendations for a Few Books of Choice

The 'books_data.edgelist' has conveniently already calculated the distance between items for you. Given this preprocessed and data, it's time to employ collaborative filtering to generate recommendations! Generate the top 10 recommendations for each book in the subset you chose. Be sure to print the book name that you are generating recommendations for as well as the name of the books being recommended.

In [22]:
#Your code here
rec_dict = {}
id_name_dict = dict(zip(books.ASIN, books.Title))
for row in HP.index:
    book_id = HP.ASIN[row]
    book_name = id_name_dict[book_id]
    most_similar = df[(df.book_1==book_id)
                      | (df.book_2==book_id)
                     ].sort_values(by='weight', ascending=False).head(10)
    most_similar['source_name'] = most_similar['book_1'].map(id_name_dict)
    most_similar['target_name'] = most_similar['book_2'].map(id_name_dict)
    recommendations = []
    for row in most_similar.index:
        if most_similar.book_1[row] == book_id:
            recommendations.append((most_similar.book_2[row], most_similar.weight[row]))
        else:
            recommendations.append((most_similar.source_name[row], most_similar.weight[row]))
    rec_dict[book_name] = recommendations
    print("Recommendations for:", book_name)
    for r in recommendations:
        print(r)
    print('\n\n')

Recommendations for: Harry Potter e la Pietra Filosofale (Italian Edition of Harry Potter and the Sorcerer's Stone)



Recommendations for: Harry Potter and the Goblet of Fire (Book 4)
('0439136350', 0.81)
('043935806X', 0.81)
('0439064864', 0.81)
('0590353403', 0.76)



Recommendations for: Pokemon & Harry Potter: A Fatal Attraction



Recommendations for: Harry Potter and the Order of the Phoenix (Book 5, Audio)
('Harry Potter and the Prisoner of Azkaban (Book 3)', 0.79)
('Harry Potter and the Chamber of Secrets (Book 2)', 0.79)
('0439139600', 0.76)
("Harry Potter and the Sorcerer's Stone (Book 1)", 0.74)



Recommendations for: Harry Potter and the Sorcerer's Stone 2002 Calendar
('0740725947', 0.0)
('0740736698', 0.0)
('0439286239', 0.0)



Recommendations for: Ultimate Unofficial Guide to the Mysteries of Harry Potter
('0972322108', 0.73)
('0425187012', 0.71)
('0970844204', 0.71)
('0613495772', 0.64)
('0826452329', 0.64)
('0439284031', 0.59)
('0767908473', 0.48)
('0786708905', 0.48

Recommendations for: Harry Potter und die Kammer des Schreckens
('3551551693', 1.0)
('3551551677', 0.77)
('3551551936', 0.77)
('501 German Verbs', 0.18)



Recommendations for: Harry Potter Stationery Kit
('Harry Potter Schoolbooks Box Set: Two Classic Books from the Library of Hogwarts School of Witchcraft and Wizardry', 0.0)
('Harry Potter Journal Box Set (3 journals)', 0.0)
('Harry Potter Journal: Hogwarts Crests', 0.0)
('Hogwarts Journal', 0.0)
('0439425212', 0.0)



Recommendations for: Harry Potter and the Sorcerer's Stone (Book 1)
('Harry Potter and the Goblet of Fire (Book 4)', 0.96)
('Harry Potter and the Prisoner of Azkaban (Book 3)', 0.92)
('Harry Potter and the Order of the Phoenix (Book 5)', 0.92)
('Harry Potter and the Chamber of Secrets (Book 2)', 0.92)



Recommendations for: Harry Potter and the Chamber of Secrets Art Coloring Book: Trace a Scene
('Harry Potter Art Coloring Book #4 (Harry Potter)', 1.0)
('Harry Potter Art Coloring Book #3 (Harry Potter)', 1.0)
("Harry 

Recommendations for: The Ivory Tower and Harry Potter: Perspectives on a Literary Phenomenon
("Harry Potter's World: Multidisciplinary Critical Perspectives (Pedagogy and Popular Culture)", 0.55)



Recommendations for: Harry Potter and the Chamber of Secrets: Deluxe Journal (Harry Potter)
('Harry Potter Hogwarts School: A Magical 3-D Carousel', 0.4)
('Harry Potter Journal Box Set (3 journals)', 0.26)
('Harry Potter Schoolbooks Box Set: Two Classic Books from the Library of Hogwarts School of Witchcraft and Wizardry', 0.19)
('Harry Potter Journal: Hogwarts Crests', 0.0)
('Harry Potter Stationery Kit', 0.0)
('0439236541', 0.0)
('043923655X', 0.0)



Recommendations for: Harry Potter and the Sorcerer's Stone (Book 1 Audio CD)
('Harry Potter and the Prisoner of Azkaban (Book 3)', 0.79)
('Harry Potter and the Order of the Phoenix (Book 5)', 0.79)
('Harry Potter and the Chamber of Secrets (Book 2)', 0.79)
('Harry Potter and the Goblet of Fire (Book 4)', 0.76)



Recommendations for: Harry P

Recommendations for: Harry Potter and the Sorcerer's Stone Movie Poster Book
('0439425220', 0.57)
('Harry Potter 2004 Day-To-Day Calendar', 0.41)
('1584152508', 0.16)
('0439286166', 0.14)
('0439288568', 0.07)
("Harry Potter and the Sorcerer's Stone 2002 Calendar", 0.0)
('0740717553', 0.0)



Recommendations for: The Gospel According to Harry Potter: Spirituality in the Stories of the World's Most Famous Seeker
('Harry Potter and the Bible: The Menace Behind the Magick (And the Bible Series)', 0.67)
('1578564719', 0.67)
('A Charmed Life : The Spirituality of Potterworld', 0.65)
('0312308698', 0.53)
('0664222226', 0.26)
('The Gospel According to the Simpsons: Leaders Guide for Group Study', 0.17)
('0664251528', 0.17)
('0664224199', 0.16)



Recommendations for: Harry Potter and the Bible: The Menace Behind the Magick (And the Bible Series)
('1578564719', 0.75)
('0664226019', 0.67)
('0312308698', 0.6)
('0875099750', 0.43)
("Alternative Worlds in Fantasy Fiction (Contemporary Classics of C

Recommendations for: Harry Potter and the Chamber of Secrets
('Harry Potter and the Order of the Phoenix (Book 5)', 0.96)
('Harry Potter and the Goblet of Fire (Book 4)', 0.96)
('Harry Potter Schoolbooks Box Set: Two Classic Books from the Library of Hogwarts School of Witchcraft and Wizardry', 0.83)



Recommendations for: Literature Guide: Harry Potter and the Prisoner of Azkaban (Grades 4-8)
('Literature Guide: Harry Potter and the Chamber of Secrets (Grades 4-8)', 1.0)
('0439211166', 1.0)
('0439231949', 1.0)
('Harry Potter and the Prisoner of Azkaban (Book 3)', 0.26)
('0439139597', 0.26)



Recommendations for: Harry Potter and the Chamber of Secrets (Book 2)
('Harry Potter and the Order of the Phoenix (Book 5)', 0.88)
('Harry Potter and the Goblet of Fire (Book 4)', 0.88)
('Harry Potter Schoolbooks Box Set: Two Classic Books from the Library of Hogwarts School of Witchcraft and Wizardry', 0.77)



Recommendations for: Harry Potter and the Chamber of Secrets 2003 Mini Wall Calendar

Recommendations for: Harry Potter and the Chamber of Secrets (Thorndike Press Large Print Young Adult Series)
('Harry Potter and the Order of the Phoenix (Book 5)', 0.89)
('Harry Potter and the Goblet of Fire (Book 4)', 0.89)
('Harry Potter Schoolbooks Box Set: Two Classic Books from the Library of Hogwarts School of Witchcraft and Wizardry', 0.84)



Recommendations for: Ultimate Unofficial Guide to the Mysteries of Harry Potter
("The Sorcerer's Companion: A Guide to the Magical World of Harry Potter", 0.5)
('Harry Potter Schoolbooks Box Set: Two Classic Books from the Library of Hogwarts School of Witchcraft and Wizardry', 0.33)



Recommendations for: Harry Potter Schoolbooks: Quidditch Through the Ages and Fantastic Beasts and Where to Find Them
('Ultimate Unofficial Guide to the Mysteries of Harry Potter', 0.59)
("The Sorcerer's Companion: A Guide to the Magical World of Harry Potter", 0.41)



Recommendations for: Harry Potter and the Sorcerer's Stone: A Deluxe Pop-up Book
('Harr

## Summary

Well done! In this lab, you effectively created a recommendation system for a real world dataset!